# Before running this notebook, ensure the following steps are completed:

* Docker Compose File: Upload the docker-compose.yml file to the root directory of your project.

* Data Directory and SQL File: Create a directory named data in the root of your project. Place the northwind.sql file inside this newly created data directory.


In [11]:
# Stop any currently running Docker Compose instances associated with this project and then restart 
# Docker Compose to load the database into PostgreSQL.

!docker compose down
!docker compose up -d

WARN[0000] /home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 0/0
 ⠋ Container indicium_tech_code_challenge-db-1  Stopping                   0.0s 
[+] Running 0/1
 ⠙ Container indicium_tech_code_challenge-db-1  Stopping                   0.1s 
[+] Running 0/1
 ⠹ Container indicium_tech_code_challenge-db-1  Stopping                   0.2s 
[+] Running 0/1
 ⠸ Container indicium_tech_code_challenge-db-1  Stopping                   0.3s 
[+] Running 0/1
 ⠼ Container indicium_tech_code_challenge-db-1  Removing                   0.4s 
[+] Running 2/1
 ✔ Container indicium_tech_code_challenge-db-1   Removed                   0.5s 
 ! Network indicium_tech_code_challenge_default  Resource is still in use  0.0s 
WARN[0000] /home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/docker-compose.yml: the attribute `version` is obsolete

In [13]:
# create a meltano project called meltano

!meltano init meltano

2025-02-04T04:03:14.447550Z [warning  ] Found meltano project at: /home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/meltano
Need help fixing this problem? Visit http://melta.no/ for troubleshooting steps, or to
join our friendly Slack community.

A `meltano.yml` file already exists in the target directory. Use `--force` to overwrite it.


In [6]:
#cd to the meltano folder and install the extractor/loaders

%cd meltano
!meltano add extractor tap-postgres
!meltano add extractor tap-csv
!meltano add loader target-csv
!meltano add loader target-postgres

[Errno 2] No such file or directory: 'meltano'
/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/meltano


/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


Added loader 'target-postgres' to your project
Variant:	meltanolabs (default)
Repository:	https://github.com/MeltanoLabs/target-postgres
Documentation:	https://hub.meltano.com/loaders/target-postgres--meltanolabs

2025-02-04T03:17:49.322795Z [info     ] Installing loader 'target-postgres'
2025-02-04T03:18:15.750158Z [info     ] Installed loader 'target-postgres'

To learn more about loader 'target-postgres', visit https://hub.meltano.com/loaders/target-postgres--meltanolabs


In [3]:
#configure the tap-postgres with the server information

!meltano config tap-postgres set database northwind
!meltano config tap-postgres set host localhost
!meltano config tap-postgres set port 5432
!meltano config tap-postgres set user northwind_user

2025-02-02T12:50:25.503419Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Extractor 'tap-postgres' setting 'database' was set in `meltano.yml`: 'northwind'
2025-02-02T12:50:28.796362Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Extractor 'tap-postgres' setting 'host' was set in `meltano.yml`: 'localhost'
2025-02-02T12:50:31.684483Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Extractor 'tap-postgres' setting 'port' was set in `meltano.yml`: 5432
2025-02-02T12:50:36.913430Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the 

### on the meltano folder, create .env with a variable
TAP_POSTGRES_PASSWORD='{database-password}'

In [ ]:
# use the .env pasword to create a schema on the DB to recive the processed data

import subprocess
from dotenv import load_dotenv
import os

load_dotenv()

# Get the password from the environment
password = os.getenv("TAP_POSTGRES_PASSWORD")

command = f"psql -h localhost -U northwind_user -d northwind -c \"CREATE SCHEMA IF NOT EXISTS golden;\""

env = os.environ.copy()
env["PGPASSWORD"] = password

try:
    subprocess.run(command, shell=True, env=env, check=True)
    print("Schema created successfully!")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")



# also, use the same passsword to configure the meltano acces to the db

command = f"meltano config tap-postgres set password {password}"
try:
    subprocess.run(command, shell=True, check=True)
    print("Password set successfully in Meltano.")
except subprocess.CalledProcessError as e:
    print(f"Error executing Meltano command: {e}")


command = f"meltano config target-postgres set password {password}"
try:
    subprocess.run(command, shell=True, check=True)
    print("Password set successfully in Meltano.")
except subprocess.CalledProcessError as e:
    print(f"Error executing Meltano command: {e}")

NOTICE:  schema "golden" already exists, skipping


CREATE SCHEMA
Schema created successfully!


2025-02-04T04:28:34.941359Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.


Extractor 'tap-postgres' setting 'password' was set in `.env`: (redacted)
Current value is still: (redacted) (from the environment)
Password set successfully in Meltano.


2025-02-04T04:28:37.702871Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.


Loader 'target-postgres' setting 'password' was set in `.env`: (redacted)
Password set successfully in Meltano.


In [17]:
!meltano config target-postgres set database northwind
!meltano config target-postgres set host localhost
!meltano config target-postgres set port 5432
!meltano config target-postgres set user northwind_user
!meltano config target-postgres set default_target_schema golden

2025-02-04T04:30:15.377493Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Loader 'target-postgres' setting 'database' was set in `meltano.yml`: 'northwind'
2025-02-04T04:30:18.298739Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Loader 'target-postgres' setting 'host' was set in `meltano.yml`: 'localhost'
2025-02-04T04:30:21.072259Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Loader 'target-postgres' setting 'port' was set in `meltano.yml`: 5432
2025-02-04T04:30:23.668136Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the 

### On the Data folder upload the order_datails.csv file provided

In [19]:
#configure the tap-csv with the path and file information

!meltano config tap-csv set files '[{"entity": "order_details", "path": "/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/order_details.csv", "keys": ["order_id"], "delimiter": ",", "encoding": "utf-8", "header": true}]'

2025-02-04T05:53:44.786561Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Extractor 'tap-csv' setting 'files' was set in `meltano.yml`: [{'entity': 'order_details', 'path': '/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/order_details.csv', 'keys': ['order_id'], 'delimiter': ',', 'encoding': 'utf-8', 'header': True}]


In [37]:
#configure the target-csv with the path destination and file name schema

!meltano config target-csv set destination_path /home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/uploaded/raw/postgres/
!meltano config target-csv set file_naming_scheme raw_{stream_name}_{datestamp}.csv

Loader 'target-csv' already exists in your Meltano project
To add it to your project another time so that each can be configured differently,
add a new plugin inheriting from the existing one with its own unique name:
	meltano add loader target-csv--new --inherit-from target-csv

2025-02-03T05:21:04.730577Z [info     ] Installing loader 'target-csv'
2025-02-03T05:21:09.113581Z [info     ] Installed loader 'target-csv' 

To learn more about loader 'target-csv', visit https://hub.meltano.com/loaders/target-csv--meltanolabs
2025-02-03T05:21:11.664406Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Loader 'target-csv' setting 'destination_path' was set in `meltano.yml`: '/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/uploaded/raw'
2025-02-03T05:21:14.412800Z [info     ] The default environment 'dev' will be ignored for `meltano config`. T

In [20]:
!meltano add extractor tap-csv--golden --inherit-from tap-csv
!meltano config tap-csv--golden set files '[{"entity": "order-merged-detail", "path": "/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/uploaded/to_go/order-merged-detail.csv", "keys": ["order_id"], "delimiter": ",", "encoding": "utf-8", "header": true}]'

Extractor 'tap-csv--golden' already exists in your Meltano project
To add it to your project another time so that each can be configured differently,
add a new plugin inheriting from the existing one with its own unique name:
	meltano add extractor tap-csv--golden--new --inherit-from tap-csv--golden

2025-02-04T05:54:31.397125Z [info     ] Installing extractor 'tap-csv--golden'
2025-02-04T05:54:36.481403Z [info     ] Installed extractor 'tap-csv--golden'

To learn more about extractor 'tap-csv--golden', visit https://hub.meltano.com/extractors/tap-csv--meltanolabs
2025-02-04T05:54:38.862430Z [info     ] The default environment 'dev' will be ignored for `meltano config`. To configure a specific environment, please use the option `--environment=<environment name>`.
Extractor 'tap-csv--golden' setting 'files' was set in `meltano.yml`: [{'entity': 'order-merged-detail', 'path': '/home/tigureis/DNC_engenharia_de_dados/Indicium_Tech_Code_Challenge/data/uploaded/to_go/order-merged-detail.csv'

In [4]:
!meltano install

2025-02-03T21:38:29.123828Z [info     ] Installing 5 plugins          
2025-02-03T21:38:29.369443Z [info     ] Installing extractor 'tap-postgres'
2025-02-03T21:38:29.445450Z [info     ] Installing extractor 'tap-csv'
2025-02-03T21:38:29.510385Z [info     ] Installing loader 'target-csv'
2025-02-03T21:38:29.541207Z [info     ] Installing orchestrator 'airflow'
2025-02-03T21:38:29.582285Z [info     ] Installing file bundle 'files-airflow'
2025-02-03T21:38:34.000012Z [info     ] Installed orchestrator 'airflow'
2025-02-03T21:38:37.840434Z [info     ] Installed loader 'target-csv' 
2025-02-03T21:38:37.866217Z [info     ] Installed extractor 'tap-csv' 
2025-02-03T21:38:40.222449Z [info     ] Installed file bundle 'files-airflow'
2025-02-03T21:38:40.253433Z [info     ] Run `meltano upgrade files` to update your project's 'files-airflow' files.
2025-02-03T21:38:41.051086Z [info     ] Installed extractor 'tap-postgres'
2025-02-03T21:38:41.051740Z [info     ] Installed 5/5 plugins         
